In [2]:
from ultralytics import YOLOv10, RTDETR, YOLO
import os
import csv
from pathlib import Path

d:\Programs\Anaconda\envs\yolov10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PATH_YOLOV10 = Path('yolov10-aug.pt')
PATH_YOLOV8 = Path('yolov8-aug.pt')
PATH_DETR = Path('detr-aug.pt')
IMAGE_PATH = ...

In [ ]:
model_yolov10 = YOLOv10(PATH_YOLOV10)
model_yolov8 = YOLO(PATH_YOLOV8)
model_DETR = RTDETR(PATH_DETR)
model_yolov10.to('cuda')
model_yolov8.to('cuda')
model_DETR.to('cuda')

In [6]:
def make_sample_submission(model, images_path:Path, conf, modelname):
    header = ['filename', 'class_id', 'rel_x', 'rel_y', 'width', 'height']
    arr = []

    filename = f'submission_{modelname}_{conf}.csv'

    results = model.predict(images_path, conf=conf, stream=True)
    for image in results:
        name = image.path.split('\\')[-1] if os.name == 'nt' else image.path.split('/')[-1]
        for box in image.boxes:
            label = int(box.cls.item())
            rel_x, rel_y, width, height = box.xywhn[0].detach().cpu().numpy()
            arr.append((name, label, rel_x, rel_y, width, height))
        
    
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')

        # Write the header to the CSV file
        writer.writerow(header)
        for row in arr:
            writer.writerow(row)

In [ ]:
conf = 0.5 #For tests

make_sample_submission(model_yolov10,IMAGE_PATH, conf, 'yolo10')
make_sample_submission(model_yolov8,IMAGE_PATH, conf, 'yolo8')
make_sample_submission(model_DETR,IMAGE_PATH, conf, 'detr') 